In [25]:
from google.colab import drive
drive.mount('/content/drive')
data_root = 'content/drive/My Drive/ChatBot'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [29]:
with open('intents.json', 'r') as file:
    # Step 2: Read the JSON data
    data = json.load(file)

In [31]:
data

{'intents': [{'tag': 'hello',
   'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"],
   'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'],
   'context': ''},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'job',
   'patterns': ['What is your job', 'What is your work'],
   'responses': ['My job is to make you feel like everything is okay.',
    'I work to serve you as well as possible'],
   'context': ''},
  {'tag': 'age',
   'patterns': ['What is your age', 'How old are you', 'When were you born'],
   'responses': ['I was born in 2021'],
   'context': ''},
  {'tag': 'feeling',
   'patterns': ['How are you today', 'How are you'],
   'responses': ['I am feeling good, you?',
    'Very good and you?',
    "Actually, I'm okay and you?"],
   'context': ''},
  {'tag': 'good',
   'patterns': ['I am good too',
    'I feel fine',
    'Good !',
  

In [5]:
!python -m chatterbot --version

/usr/bin/python3: No module named chatterbot


In [7]:
!git clone https://github.com/gunthercox/ChatterBot.git

Cloning into 'ChatterBot'...
remote: Enumerating objects: 12870, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 12870 (delta 16), reused 19 (delta 8), pack-reused 12837
Receiving objects: 100% (12870/12870), 4.25 MiB | 9.36 MiB/s, done.
Resolving deltas: 100% (8954/8954), done.


In [32]:
#creating data_x and data_y
words = []
classes = []
data_X = []
data_y = []


for intent in data['intents']:
  for pattern in intent['patterns']:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_X.append(pattern)
    data_y.append(intent['tag'])
  if intent['tag'] not in classes:
    classes.append(intent['tag'])

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [34]:
training = []
out_empty = [0]*len(classes)

for idx,doc in enumerate(data_X):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words :
    bow.append(1) if word in text else bow.append(0)
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1
  training.append([bow,output_row])
random.shuffle(training)
training = np.array(training , dtype = object)
train_X = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

## Building the Neural network model

In [38]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_X[0]),),activation  = 'relu'))
model.add(Dropout(0,5))
model.add(Dense(64,activation  = 'relu'))
model.add(Dropout(0,5))
model.add(Dense((len(train_y[0])),activation  = 'softmax'))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['accuracy']
              )
print(model.summary())
model.fit(x=train_X,y=train_y,epochs = 150,verbose = 1)





Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               7552      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
Total params: 16848 (65.81 KB)
Trainable params: 16848 (65.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2

#user data preprocessing

In [45]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text,vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens :
    for idx,word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list
def get_response(intents_list,intents_json):
  if len(intents_list) == 0 :
    result = "sorry i dont understand"
  else :
    tag=intents_list[0]
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
      if i['tag'] == tag:
        result = random.choice(i["responses"])
        break
  return result


In [46]:
print("Press 0 if you don't want to chat with our Chatbot")
while True :
  message = input('')
  if message == '0':
    break
  intents = pred_class(message,words,classes)
  result = get_response(intents,data)
  print(result)

Press 0 if you don't want to chat with our Chatbot

1/1 [==============================] - 0s 22ms/step
So, everything's okay!
yes
1/1 [==============================] - 0s 31ms/step
That is perfect!
whats your name
1/1 [==============================] - 0s 21ms/step
That is perfect!
ok
1/1 [==============================] - 0s 23ms/step
That is perfect!
exit
1/1 [==============================] - 0s 44ms/step
So, everything's okay!
nice
1/1 [==============================] - 0s 23ms/step
That is perfect!


KeyboardInterrupt: Interrupted by user